# Milestone 1：快速核对

目标：读取 1 名受试者（默认 28），导出 10 秒连续片段与反馈事件概览，并记录反馈事件计数。
注意：函数都在 `src/` 中实现，Notebook 仅做调用与展示。

In [ ]:
# 将仓库根目录加入路径，便于导入 src 模块
import sys
from pathlib import Path
ROOT = Path.cwd()
while ROOT != ROOT.parent and not (ROOT / 'src').exists():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))
print('Repo root:', ROOT)

In [ ]:
# 导入依赖与项目函数
import mne
from IPython.display import Image, display
from src.config import ensure_dirs, FIGURES_DIR, RESULTS_DIR
from src.preprocessing import read_raw_bids, PreprocParams, preprocess_raw
from src.events import get_feedback_events
SUBJECT = '28'  # 可修改为其他编号
ensure_dirs()

In [ ]:
# 1) 读取并进行轻量滤波（仅用于核对可视化）
raw = read_raw_bids(SUBJECT)
raw_p = preprocess_raw(raw, PreprocParams(l_freq=1.0, h_freq=40.0, notch=50.0, use_icalabel=False))

# 连续片段图
fig1 = raw_p.plot(duration=10.0, n_channels=32, show=False)
p1 = FIGURES_DIR / f'm1_continuous_{SUBJECT}.png'
fig1.savefig(p1, dpi=150)
display(Image(filename=str(p1)))

In [ ]:
# 2) 反馈事件概览（仅 win/loss，合并 high/mid/low）
events, event_id = get_feedback_events(raw)
fig2 = mne.viz.plot_events(events, event_id=event_id, show=False)
p2 = FIGURES_DIR / f'm1_events_{SUBJECT}.png'
fig2.savefig(p2, dpi=150)
display(Image(filename=str(p2)))

# 3) 写入简要报告
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
counts = {name: int((events[:, 2] == code).sum()) for name, code in event_id.items()}
report = RESULTS_DIR / f'm1_summary_{SUBJECT}.txt'
with open(report, 'w', encoding='utf-8') as f:
    f.write(f'Subject: {SUBJECT}\n')
    f.write('Event counts (feedback only):\n')
    for k in sorted(counts):
        f.write(f'  {k}: {counts[k]}\n')
    f.write(f'Figures:\n  {p1}\n  {p2}\n')
print('Saved report:', report)